In [31]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
import csv
from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()

import tensorflow as tf

In [32]:
def remove_stop_words(text: string):
    return ' '.join([item for item in text.split() if item not in stopwords.words('english')])

def remove_punctuations(text: string):
    return "".join([char for char in text if char not in string.punctuation])

def stem_text(text: string):
    return [ps.stem(word) for word in text]

def delete_usernames(text: str) -> str:
    pattern = r'@\w+'
    return re.sub(pattern, '@user', text)

In [33]:
before_corpus = ""
after_corpus = ""

with open("Pre-processing.csv", 'r', newline='') as csvFile:
    reader = csv.reader(csvFile)
    next(reader) #skip header
    for line in reader:
        before_corpus+=line[3].strip()
        after_corpus+=line[4].strip()

before_corpus = delete_usernames(before_corpus)
print(before_corpus)

Just over a month ago, the train derailment in East Palestine, Ohio caused metric tons of toxic chemicals to spew into the air and water.   Many locals are already suffering health issues as a result, and tens of thousands pets and wildlife have been reported dead.I am infinitely more concerned with the raging chemical fire in East Palestine OH then I am with NORAD playing whack-a-mole with balloons.Jew here.  Words mean things.  When everything is called antisemitic, it helps provide cover for actual antisemitism.If you were on a mission to create more antisemitism in this country, you couldn’t do a better job than the ADL.Isn’t it crazy that almost no one in the political or media class talk about radical Islamic terrorism anymore. There’s no concern about it, and yet every emergency power that the government took in the name of terrorism, DHL, TSA, Patriot act, is just part of America now.You cannot possibly compare NATOs interventions to Russias invasions. It requires both a delibe